# Data Quality Checks: Completeness, Uniqueness, Validity

**Note:** The column `Index (Row number)` is used only as a reference and is excluded from all checks.

In [77]:
import pandas as pd

# Load the CSV file
file_path = 'Planview Data.csv'
df = pd.read_csv(file_path)
df.head()

C:\Users\uzair\AppData\Local\Temp\ipykernel_5844\1156686937.py:5: DtypeWarning: Columns (21,22,24,34,35,44,45,47,56,93,100,112,114,116,117,121,123,124,130,133,134,135,136,137,138,149,150,154,155,156,157,162,164,165,169,170,171,172,173,174,177,178,179,181,182,185,186,187,194,195,196,198,199,200,201,202,205,208,209,210,211,219,221,224,225) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Index (Row number),wbs_lvl_8,Project_Name,Work_ID,Work_Type,wbs_lvl_4,wbs_lvl_5,wbs_lvl_3,wbs_lvl_2,Project_Category,...,What_Worked_LL.1,Change_Experience_and_Acceptance_into_Service_LL,Change_Experience_and_Acceptance_into_Service_LL.1,Metrics_and_Feedback_LL,Metrics_and_Feedback_LL.1,Human_Factors_LL,Human_Factors_LL.1,Project_Execution_LL,Project_Planning_LL,Work_Status.1
0,1,Nationwide Building Society,NBS00074753 - Admin WIFI Remediation,1040466,Customer Implementation,Global,Global,Global,Business,In-Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed
1,2,Rolls Royce,GSM00005487-GWAN-Modify-MONTREAL BORON 9500,1041114,Customer Implementation,UK MNC,MNC Manufacturing,CPS,Business,In-Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed
2,3,Rolls Royce,GSM00005478 - All IP Professional Services,1041090,Customer Implementation,UK MNC,MNC Manufacturing,CPS,Business,In-Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open/Active
3,4,3M,3M - APAC - T&T - Arindam Roy,1012006,Customer Implementation,Manufacturing,Life Sciences & Technology,Global,Business,T&T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed
4,5,3M,3M - Americas - T&T - Franklin Preciado,1012007,Customer Implementation,Manufacturing,Life Sciences & Technology,Global,Business,T&T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed


## Task 1: Completeness Check (Excludes 'Index (Row number)')

In [78]:
# Drop the index column for quality checks
columns_to_check = df.columns.drop("Index (Row number)")

# Convert to long format and check completeness
df_long = df.reset_index().melt(id_vars=["Index (Row number)"], value_vars=columns_to_check,
                                var_name="Column Name", value_name="Value")
df_long["Complete"] = df_long["Value"].notna() & (df_long["Value"].astype(str).str.strip() != "")
df_long["Complete"] = df_long["Complete"].astype(int)

# Base output structure
df_output = df_long[["Column Name", "Index (Row number)", "Complete"]].copy()
df_output["Unique"] = None
df_output["Validity"] = None

## Task 2: Uniqueness Check

In [79]:
uniqueness_columns = ["Work_ID", "JA_Code_Task_ID", "semblance_ID", "CRI_ID", "CRI_ID.1", "CRI_ID.2"]
non_unique_entries = []

for col in uniqueness_columns:
    if col in df.columns:
        col_values = df[col]
        value_counts = col_values.value_counts(dropna=False)
        unique_flags = col_values.map(lambda x: 1 if value_counts[x] == 1 else 0)

        mask = df_output["Column Name"] == col
        df_output.loc[mask, "Unique"] = unique_flags.values

        # Collect non-unique entries
        non_unique_mask = col_values.isin(value_counts[value_counts > 1].index)
        non_unique_df = df.loc[non_unique_mask, ["Index (Row number)", col]].copy()
        non_unique_df["Column Name"] = col
        non_unique_df.rename(columns={col: "Value"}, inplace=True)
        non_unique_entries.append(non_unique_df)

non_unique_results = pd.concat(non_unique_entries, ignore_index=True)

## Task 3: Validation Check

In [80]:
valid_work_status = {"Open/Active", "Completed", "Cancelled", "Denied", "New/Requested"}
valid_complexity = {"Low", "Medium", "High"}
valid_colors = {"Amber", "Blue", "Red", "Green"}

# Work_Status
if "Work_Status.1" in df.columns:
    valid_mask = df["Work_Status.1"].isin(valid_work_status)
    df_output.loc[df_output["Column Name"] == "Work_Status.1", "Validity"] = valid_mask.astype(int).values

# Complexity
if "Complexity" in df.columns:
    valid_mask = df["Complexity"].isin(valid_complexity)
    df_output.loc[df_output["Column Name"] == "Complexity", "Validity"] = valid_mask.astype(int).values

# Columns AJ to AO (columns 35 to 40)
aj_to_ao_cols = df.columns[35:41]
for col in aj_to_ao_cols:
    if col in df.columns:
        valid_mask = df[col].isin(valid_colors)
        df_output.loc[df_output["Column Name"] == col, "Validity"] = valid_mask.astype(int).values

df_output["Validity"] = df_output["Validity"].where(df_output["Validity"].notna(), None)

## Adding Work_ID, wbs_lvl_3 and Project_Category to the output

In [81]:
import pandas as pd

# Load Planview data
planview_df = pd.read_csv("Planview Data.csv")

# Ensure 'Index (Row number)' exists
if 'Index (Row number)' in planview_df.columns:
    planview_df.set_index('Index (Row number)', inplace=True)

# If df_output does not already contain the correct row index, extract it
if 'Index (Row number)' not in df_output.columns:
    # If original row indices were preserved somewhere, use that. Otherwise:
    df_output['Index (Row number)'] = df_output.index + 1

# Safely extract the values using .get with fallback
def get_field_value(i, field):
    try:
        return planview_df.at[i, field] if i in planview_df.index else ''
    except KeyError:
        return ''

# Populate the columns using the correct index
df_output['Work_ID'] = df_output['Index (Row number)'].apply(lambda i: get_field_value(i, 'Work_ID'))
df_output['wbs_lvl_3'] = df_output['Index (Row number)'].apply(lambda i: get_field_value(i, 'wbs_lvl_3'))
df_output['Project_Category'] = df_output['Index (Row number)'].apply(lambda i: get_field_value(i, 'Project_Category'))

# Reorder columns
desired_order = ['Column Name', 'Index (Row number)', 'Work_ID', 'wbs_lvl_3', 'Project_Category', 'Complete', 'Unique', 'Validity']
df_output = df_output[[col for col in desired_order if col in df_output.columns]]

# Preview
print(df_output.head(10))


C:\Users\uzair\AppData\Local\Temp\ipykernel_5844\2049552260.py:4: DtypeWarning: Columns (21,22,24,34,35,44,45,47,56,93,100,112,114,116,117,121,123,124,130,133,134,135,136,137,138,149,150,154,155,156,157,162,164,165,169,170,171,172,173,174,177,178,179,181,182,185,186,187,194,195,196,198,199,200,201,202,205,208,209,210,211,219,221,224,225) have mixed types. Specify dtype option on import or set low_memory=False.
  planview_df = pd.read_csv("Planview Data.csv")


  Column Name  Index (Row number)  Work_ID wbs_lvl_3 Project_Category  \
0   wbs_lvl_8                   1  1040466    Global          In-Life   
1   wbs_lvl_8                   2  1041114       CPS          In-Life   
2   wbs_lvl_8                   3  1041090       CPS          In-Life   
3   wbs_lvl_8                   4  1012006    Global              T&T   
4   wbs_lvl_8                   5  1012007    Global              T&T   
5   wbs_lvl_8                   6  1012008    Global              T&T   
6   wbs_lvl_8                   7  1012009    Global          In-Life   
7   wbs_lvl_8                   8  1012010    Global          In-Life   
8   wbs_lvl_8                   9  1012011    Global              T&T   
9   wbs_lvl_8                  10  1012013    Global              T&T   

   Complete Unique Validity  
0         1   None     None  
1         1   None     None  
2         1   None     None  
3         1   None     None  
4         1   None     None  
5         1   No

## Addind M/SM/Required and Header in the final output

In [82]:
# Load Fields features data
fields_df = pd.read_csv("Fields features.csv")

# Create a mapping from 'Field Name' to 'M/SM/Required' and 'Header'
fields_df.set_index('Field Name', inplace=True)

# Match each Column Name in df_output to the features
df_output['M/SM/Required'] = df_output['Column Name'].map(
    lambda x: fields_df.at[x, 'M/SM/Required'] if x in fields_df.index else '')
df_output['Header'] = df_output['Column Name'].map(
    lambda x: fields_df.at[x, 'Header'] if x in fields_df.index else '')

# Reorder columns as requested
final_order = ['Column Name', 'Index (Row number)', 'Work_ID', 'wbs_lvl_3', 'Project_Category',
               'M/SM/Required', 'Header', 'Complete', 'Unique', 'Validity']
df_output = df_output[[col for col in final_order if col in df_output.columns]]

# Preview
print(df_output.head(10))


  Column Name  Index (Row number)  Work_ID wbs_lvl_3 Project_Category  \
0   wbs_lvl_8                   1  1040466    Global          In-Life   
1   wbs_lvl_8                   2  1041114       CPS          In-Life   
2   wbs_lvl_8                   3  1041090       CPS          In-Life   
3   wbs_lvl_8                   4  1012006    Global              T&T   
4   wbs_lvl_8                   5  1012007    Global              T&T   
5   wbs_lvl_8                   6  1012008    Global              T&T   
6   wbs_lvl_8                   7  1012009    Global          In-Life   
7   wbs_lvl_8                   8  1012010    Global          In-Life   
8   wbs_lvl_8                   9  1012011    Global              T&T   
9   wbs_lvl_8                  10  1012013    Global              T&T   

  M/SM/Required Header  Complete Unique Validity  
0           NaN    NaN         1   None     None  
1           NaN    NaN         1   None     None  
2           NaN    NaN         1   None    

## Split Output into 4 Parts

In [84]:
total_rows = len(df_output)
split_size = total_rows // 4
for i in range(4):
    start_idx = i * split_size
    end_idx = (i + 1) * split_size if i < 3 else total_rows
    part_df = df_output.iloc[start_idx:end_idx]
    part_df.to_csv(f"final_output_part_{i + 1}.csv", index=False)

## Save Non-Unique Entries (Optional)

In [ ]:
non_unique_results.to_csv("non_unique_entries.csv", index=False)